In [3]:
%load_ext autoreload
%autoreload 2

from data.projectionsdataset import ProjectionsDataset
from data.emdataset import EMDataset
import matplotlib.pyplot as plt 
from tifffile import imread
from data import transforms
from PIL import Image
from tqdm import tqdm
import open3d as o3d
import numpy as np
import torchvision
import importlib
import warnings
# import mrcfile
# import astra
import torch
import os
import PIL

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
data_path = "data/Au_Ag" 
projections_path = os.path.join(data_path, "Serie_3_ali.tif")
angles_path = os.path.join(data_path, "TiltAngle_ali.txt")
mrc_path = os.path.join(data_path, "Serie_3.mrc")

In [5]:
def get_n_channels(mode):
    return {
        "1": 1,  
        "L": 1,  
        "P": 1,  
        "RGB": 3,  
        "RGBA": 4,  
        "CMYK": 4,  
        "YCbCr": 3,  
        "I": 1,  
        "F": 1,  
    }.get(mode, 1)  


def read_angles_file(filepath: str) -> torch.Tensor:
    with open(filepath, 'r') as file:
        lines = file.readlines()
    
    return torch.from_numpy( np.array([float(line.strip()) for line in lines], dtype=np.float32) )


def estimate_memory(n, size, channels=1, dtype=torch.uint8):

    bytes_per_pixel = torch.tensor([], dtype=dtype).element_size()
    
    memory_per_image = size[0] * size[1] * channels * bytes_per_pixel
    
    total_memory_bytes = n * memory_per_image
    
    print( f"Storing this object uses: {total_memory_bytes / 1024**2} MB of memory" )

def tensor_memory_usage(tensor: torch.Tensor) -> float:
    num_elements = tensor.numel()
    element_size = tensor.element_size()
    total_size_in_bytes = num_elements * element_size
    print( f"Storing this tensor uses: {total_size_in_bytes / 1024**2} MB of memory" )
    
def get_vram_info():
    total_vram = torch.cuda.get_device_properties(0).total_memory
    allocated_vram = torch.cuda.memory_allocated(0)
    free_vram = total_vram - allocated_vram
    return total_vram, allocated_vram, free_vram